# Sistemas de filas

Após uma contagem de, pelo menos, uma hora em um sistema que apresente fila de espera, preferencialmente com mais de um servidor, o aluno deverá apresentar respostas às perguntas que serão apresentadas a seguir.

In [1]:
from datetime import datetime, timedelta
from IPython.display import display
from math import factorial
from matplotlib import pyplot as plt

import io
import numpy as np
import pandas as pd

Σ = sum

%matplotlib inline

### Dados

In [2]:
def timetable(a, b):
    return b + timedelta(minutes=int(a))
    
v_timetable = np.vectorize(timetable)

In [3]:
def sampling_generator(
    size: int=20, initial_datetime: datetime=None
) -> np.array:
    """
    
    """
    if initial_datetime is None:
        initial_datetime = datetime.now()
    
    return v_timetable(
        np.random.randint(0, 60, size=size),
        datetime.now()
    )

In [4]:
def mean_diff_time(se_time: pd.Series) -> float:
    """
    
    """
    n = se_time.shape[0]
    l_diff = [0] * (n-1)
    
    for i in range(1, n):
        j = i-1
        l_diff[j] = (se_time[i] - se_time[j]).seconds/60
    
    return np.mean(l_diff)

In [5]:
initial_datetime = datetime.now()

df_chegada = pd.DataFrame({
    'entrada': sampling_generator(
        initial_datetime=initial_datetime
    )
}).sort_values(by='entrada').reset_index(drop=True)

df_saida = pd.DataFrame({
    'saída': sampling_generator(
        initial_datetime=initial_datetime
    )
}).sort_values(by='saída').reset_index(drop=True)

"""
fig, ax = plt.subplots(1, 2)
df_chegada.hist(ax=ax[0])

df_saida.hist(ax=ax[1])
plt.show()
"""
print('tables head')
display(df_chegada.head())
display(df_saida.head())

tables head


,entrada
0,2016-06-08 14:05:27.938347
1,2016-06-08 14:06:27.938347
2,2016-06-08 14:07:27.938347
3,2016-06-08 14:08:27.938347
4,2016-06-08 14:12:27.938347


,saída
0,2016-06-08 14:08:27.943735
1,2016-06-08 14:11:27.943735
2,2016-06-08 14:13:27.943735
3,2016-06-08 14:16:27.943735
4,2016-06-08 14:19:27.943735


**a) Descrição do sistema de filas, local, data e horários da coleta de dados:**

Fila do tipo: FIFO

Local: loja Plazafone

Início da coleta:

In [6]:
initial_datetime.strftime('%d de %b de %Y - %l:%M%p')

'08 de Jun de 2016 -  2:05PM'

**b) Número de servidores atendendo = S:**

In [7]:
S = 3
print('S:', S)

S: 3


**c) O tempo médio entre as chegadas à fila ( um cliente a cada 5 min, por ex.):**

In [8]:
μ_chegada = mean_diff_time(df_chegada['entrada'])
print('μ_chegada:', μ_chegada)

μ_chegada: 2.89473684211


**d) A taxa de chegada de usuários no sistema de filas – λ ( clientes por unidade de tempo):**

$\lambda = \frac{60 min}{\mu\_chegada} (clientes/hora)$

where:

* $\mu\_chegada$ é o tempo médio entre as chegadas à fila (item c).

In [9]:
λ = 60/μ_chegada
print('λ:', λ)

λ: 20.7272727273


**e) O tempo médio de atendimento de cada cliente ( 20 min, por ex.):**

In [10]:
μ_atendimento = mean_diff_time(df_saida['saída'])
print('μ_atendimento:', μ_atendimento)

μ_atendimento: 2.89473684211


**f) A taxa de atendimento dos usuários – μ ( clientes por unidade de tempo, por servidor:**

$\mu = \frac{60 min}{\mu\_atendimento m} (clientes/hora)$

Se existir mais que um servidor, a taxa de atendimento do sistema será multiplicada pelo

número de servidores : S=5, então a taxa será de 5μ=15 clientes por hora.

In [11]:
μ = 60/μ_atendimento
print('μ:', μ)

μ: 20.7272727273


**g) A ocupação do sistema(em porcentagem);**

$p = \frac{\lambda}{S*\mu} * 100 (\%)$

In [12]:
p = (λ/S*μ)*100
print('p:', p)

p: 14320.661157


h) A probabilidade de o sistema estar vazio;

$$
P_0 = [
    \sum\limits_{n=0}^{S-1} (\frac{(\lambda / \mu)^n}{n!}) + 
    \frac{(\lambda/\mu)^S}{S!(1-(\lambda/(S*\mu)))}
]^{-1}
$$

In [13]:
p0 = Σ([
    (((λ/μ)**n)/(factorial(n))) +
    (((λ/μ)**S)/(factorial(S) * (1-(λ/(S*μ)))))
    for n in range(S)
])**(-1)

print('p0:', p0)

p0: 0.307692307692


i) A probabilidade de todos os servidores estarem ocupados;

j) O número esperado de usuários no sistema;

k) O número esperado de usuários na fila;

l) O tempo médio dos usuários na fila;

m) O tempo provável dos usuários no sistema;

n) Se a taxa de chamadas duplicasse, o que aconteceria com o sistema? E quais as providências que deveriam ser tomadas.

## Referências

http://www.mathpages.com/home/kmath026/kmath026.htm

http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.random.poisson.html
